# Training the final model

This notebook is supplementary material to the project here, which aims to re-implement the Hubble.2d6 tool to predict the function of CYP2D6 star alleles.

Within this notebook, the final model in the sequence outlined in the paper is trained using transfer learning. Weights from the first step are loaded into the new model. The fully-connected layers of the first model are replaced with new, randomly initialised layers and trained on 31 star alleles and their respective suballeles.

Please keep in mind that this implementation is incomplete due to the lack of resources to compute robust annotation embeddings of all variants for all star alleles.

Additionally, in the paper, this model is supposed to instead have its weights - along with weights of one of the fully-connected layers - loaded in from the second model in the sequence. However, due to data availability, I was unable to reproduce the implementation of the second model and instead chose to transfer just the convolution layers' weights to the final model from the first model that was trained on simulated data.

## Getting ready

**Acknowledgements**: Pre-computed annotation embeddings used are form the original Hubble.2d6 repo: https://github.com/gregmcinnes/Hubble2D6/tree/master/data

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
np.random.seed(1337)

In [ ]:
!git clone https://github.com/Locrian24/seng474-term-project.git
!cd seng474-term-project/ && git pull

Cloning into 'seng474-term-project'...
remote: Enumerating objects: 95, done.
remote: Counting objects: 100% (95/95), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 95 (delta 38), reused 75 (delta 21), pack-reused 0
Unpacking objects: 100% (95/95), done.
Already up to date.


In [ ]:
import sys
sys.path.insert(0, '/content/seng474-term-project')

## Preparing the data

The vcf file of the star alleles used were provided from the paper and can be found **here**. 

### Encoding method

The script `build_label_csv.py` was run on this vcf file to output the label csv that will be used for training.

Labels for the data correspond to the ordinal classes describing the function of each CYP2D6 star allele and their respective suballeles: "No function", "Decreased function", and "Normal function".

The model however is required to output two scores, representing the probability of a star allele being "No function" and "Normal function" respectively, and therefore a binary scoring system is used to encode the 3 function classes into these two scores. 

The scoring system is as follows:
- "No function" alleles are indicated with a 0 as the first score, with all other functions being denoted with a 1
- "Normal function" alleles are indicated with a 1 as the second score, with all other functions being denoted with a 0.

This yields the following encodings:
- "No function" = `[0, 0]`
- "Decreased function" = `[0, 1]`
- "Normal function" = `[1, 1]`

**Note**: Star alleles with uncurated function have an empty label (`[None, None]`). 

_Sequence encodings are done exactly the same as in step 1_

In [ ]:
from encode_to_seq import Encode2Seq

# Global variables rn for testing

ANNOTATIONS = '/content/seng474-term-project/data/gvcf2seq.annotation_embeddings.csv'
EMBEDDINGS = '/content/seng474-term-project/data/embeddings.txt'
REF = '/content/seng474-term-project/data/ref.seq'

VCF = '/content/seng474-term-project/step3/star_samples.vcf'
LABELS = '/content/seng474-term-project/step3/labels.csv'

encoding = Encode2Seq(vcf=VCF, labels=LABELS, label_cols=[0, 1, 2], embedding_file=EMBEDDINGS, annotation_file=ANNOTATIONS, ref_seq=REF)

### Seperation of data sets

The paper indicates that they used 31 star alleles and their suballeles as training data, and 24 alleles to compose the test data. Additionally, 10% of the each functional class ("No function", "Decreased function", and "Normal function") were held out for validation during training.

All star alleles with "Uncurated function" were ignored, and held from training/testing for obvious reasons.

In [ ]:
# Select only star alleles with curated function

mask = np.all(np.isnan(encoding.y) == False, axis=1)

sample_y = encoding.y[mask]
sample_X = encoding.X[mask.reshape(-1, 1).any(axis=1)]
sample_names = encoding.sample_names[mask]

In [ ]:
# Get valid stars
all_stars = np.array([s.split('_')[1] for s in sample_names])
stars, idx = np.unique(all_stars, return_index=True)

# Choose which stars are training and which are test: (31, 24) split
# Should be stratified with labels
train_idx, test_idx = train_test_split(idx, stratify=sample_y[idx], test_size=24, random_state=0)

# Retrieve indices of training and test stars
sample_mask = np.isin(all_stars, all_stars[train_idx])

test_stars = np.array([s for s in sample_names[~sample_mask] if s.split('_')[-1] == 'vcf'])
test_mask = np.isin(sample_names, test_stars)

# Split the data into the two sets - INCLUDING SUBALLES ON BOTH
# _train_X, test_X = sample_X[sample_mask], sample_X[~sample_mask]
# _train_y, test_y = sample_y[sample_mask], sample_y[~sample_mask]

_train_X, test_X = sample_X[sample_mask], sample_X[test_mask]
_train_y, test_y = sample_y[sample_mask], sample_y[test_mask]

# Split training into train + validation (10% split -> validation)
train_X, val_X, train_y, val_y = train_test_split(_train_X, _train_y, stratify=_train_y, test_size=0.1, random_state=0)

In [ ]:
# Uncurated star alleles

uncurated_samples = encoding.sample_names[~mask]
uncurated_stars = np.array([s for s in uncurated_samples if s.split('_')[-1] == 'vcf'])
uncurated_star_mask = np.isin(uncurated_samples, uncurated_stars)

# uncurated_X = encoding.X[uncurated_star_idx]
# uncurated_samples = uncurated_samples[uncurated_star_idx]

uncurated_samples = uncurated_samples[uncurated_star_mask]
uncurated_X = encoding.X[(~mask).reshape(-1, 1).any(axis=1)]
uncurated_X = uncurated_X[uncurated_star_mask]

### Prepare data for the model

In [ ]:
_train_ds = tf.data.Dataset.from_tensor_slices((train_X, train_y))
train_ds = _train_ds.shuffle(32).batch(32).prefetch(buffer_size=10)

_val_ds = tf.data.Dataset.from_tensor_slices((val_X, val_y))
val_ds = _val_ds.shuffle(32).batch(32).prefetch(buffer_size=10)

_test_ds = tf.data.Dataset.from_tensors((test_X, test_y))
test_ds = _test_ds.prefetch(buffer_size=10)

## Building the final model

As stated earlier, the model is loaded from the generated model from step 1, and also inherits its learned weights.

In [ ]:
epochs = 20
fine_tune_epochs = 7

def build_and_fit(train_dataset, val_dataset):
  json_file = open('/content/seng474-term-project/step_1/model.json', 'r')
  loaded_model = json_file.read()
  model = tf.keras.models.model_from_json(loaded_model)
  model.load_weights('/content/seng474-term-project/step_1/weights.h5')
  
  # Remove fully connected layers
  model.pop()
  model.pop()
  model.pop()
  model.trainable = False

  # Build final model
  inputs = tf.keras.Input(shape=(14868, 13))
  # model.training = False
  x = model(inputs, training=False)
  x = tf.keras.layers.Dense(32, activation=tf.keras.activations.relu, kernel_initializer=tf.keras.initializers.VarianceScaling(mode='fan_avg', distribution='uniform'), name = "dense_5")(x)
  x = tf.keras.layers.Dropout(rate=0.03, name="dropout_4")(x)
  x = tf.keras.layers.Dense(1, activation=tf.keras.activations.linear, kernel_initializer=tf.keras.initializers.VarianceScaling(mode='fan_avg', distribution='uniform'), name = "dense_6")(x)
  outputs = tf.keras.layers.Dense(2, activation=tf.keras.activations.sigmoid, kernel_initializer=tf.keras.initializers.VarianceScaling(mode='fan_avg', distribution='uniform'), name = "final_dense")(x)
  final_model = tf.keras.Model(inputs, outputs)

  # Initial training
  final_model.compile(
    optimizer=tf.keras.optimizers.Adam(0.01),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=[tf.keras.metrics.BinaryAccuracy()]
  )

  final_model.fit(train_dataset, epochs=epochs, validation_data=val_dataset, verbose=0)

  # Fine tuning
  model.trainable = True
  # model.training = True
  final_model.compile(
      tf.keras.optimizers.Adam(1e-5),
      loss=tf.keras.losses.BinaryCrossentropy(),
      metrics=[tf.keras.metrics.BinaryAccuracy()]
  )

  final_model.fit(train_dataset, epochs=fine_tune_epochs, validation_data=val_dataset, verbose=0)
  
  return final_model

## Training ensemble models

Hubble.2d6 uses an ensemble averaging method for it's final predictions. Here we are training 7 models to use in the ensemble, report the training accuracy of each and save these models for the final tool.

In [ ]:
ensemble_size = 7

for i in range(ensemble_size):
  model = build_and_fit(train_ds, val_ds)
  model.evaluate(train_ds)
  model.evaluate(val_ds)

  model.save(f'models/ensemble_{i}.model.h5')

1/1 [==============================] - 0s 70ms/step - loss: 0.4126 - binary_accuracy: 0.8077


Now that we have the ensemble models saved, we will use them for the final prediction.

## Saving evaluation data

We are saving the test dataset for our evaluation step on the ensemble model.

In [27]:
np.savetxt("test.samples.csv", test_X.reshape(test_X.shape[0], 14868 * 13), delimiter=",")
np.savetxt("test.labels.csv", test_y, delimiter=",")